In [40]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim

import requests
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium
from folium import plugins

In [2]:
# assign the url 
url = 'https://id.wikipedia.org/wiki/Daftar_kecamatan_dan_kelurahan_di_Kota_Bandung'
req = urllib.request.urlopen(url)
article = req.read().decode()

In [3]:
# use the Beautiful Soup
soup = BeautifulSoup(article, 'html.parser')
tables = soup('table')

In [4]:
# read the headings from table
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:5] == ['Kode Kemendagri', 'Kecamatan', 'Jumlah Kelurahan', 'Status', 'Daftar Kelurahan']:
        break

print(headings)

['Kode Kemendagri', 'Kecamatan', 'Jumlah Kelurahan', 'Status', 'Daftar Kelurahan', '', 'TOTAL', '151', '', '']


In [5]:
# assign the headings
headings = ['Kode Kemendagri', 'Kecamatan','Jumlah Kelurahan','Status','Kelurahan']
df=pd.DataFrame(columns=headings)
df

,Kode Kemendagri,Kecamatan,Jumlah Kelurahan,Status,Kelurahan


In [6]:
# assign the value to the dataframe
for tr in table.find_all('tr'):
    tds = tr.find_all('td')
    if not tds:
        continue
    kode, kecamatan, jumlah_kelurahan, status, daftarkelurahan = [td.text.strip() for td in tds]
    df = df.append({'Kode Kemendagri' : kode, 'Kecamatan':kecamatan, 'Jumlah Kelurahan':jumlah_kelurahan, 'Status':status, 'Kelurahan':daftarkelurahan}, ignore_index=True)

In [7]:
df.drop('Kode Kemendagri', axis=1, inplace=True)
df.drop('Jumlah Kelurahan', axis=1, inplace=True)
df.drop('Status', axis=1, inplace=True)
df.head()

,Kecamatan,Kelurahan
0,Andir,Campaka\nCiroyom\nDunguscariang\nGaruda\nKebon...
1,Astana Anyar,Cibadak\nKaranganyar\nKarasak\nNyengseret\nPan...
2,Antapani,Antapani Kidul\nAntapani Kulon\nAntapani Tenga...
3,Arcamanik,Cisaranten Bina Harapan\nCisaranten Endah\nCis...
4,Babakan Ciparay,Babakan\nBabakanciparay\nCirangrang\nMargahayu...


In [8]:
df['Kelurahan'] = df['Kelurahan'].str.split("\n", expand=False)
df.head()

,Kecamatan,Kelurahan
0,Andir,"[Campaka, Ciroyom, Dunguscariang, Garuda, Kebo..."
1,Astana Anyar,"[Cibadak, Karanganyar, Karasak, Nyengseret, Pa..."
2,Antapani,"[Antapani Kidul, Antapani Kulon, Antapani Teng..."
3,Arcamanik,"[Cisaranten Bina Harapan, Cisaranten Endah, Ci..."
4,Babakan Ciparay,"[Babakan, Babakanciparay, Cirangrang, Margahay..."


In [9]:
new_df = df.Kelurahan.apply(pd.Series)
new_df = new_df.merge(df, left_index=True, right_index=True)
new_df.drop(["Kelurahan"], axis = 1, inplace=True)
new_df = new_df.melt(id_vars = ['Kecamatan'], value_name = "Kelurahan")
new_df.drop("variable", axis = 1, inplace=True)
new_df.dropna(inplace=True)
new_df.sort_values(by=['Kecamatan'], inplace=True)
new_df.reset_index(drop=True, inplace=True)
new_df.head()

,Kecamatan,Kelurahan
0,Andir,Campaka
1,Andir,Ciroyom
2,Andir,Garuda
3,Andir,Kebonjeruk
4,Andir,Maleber


In [10]:
coordinates = pd.read_csv('koordinat-dan-ketinggian-kantor-kelurahan-di-kota-bandung-2014.csv')
coordinates.head()

,Kecamatan,Kelurahan,Lintang Selatan,Bujur Timur,Ketinggian (dpl)
0,Bandung Kulon,Gempolsari,-6.92911,107.55907,696
1,Bandung Kulon,Cigondewah Kaler,-6.93411,107.56361,700
2,Bandung Kulon,Cigondewah Kidul,-6.94386,107.56005,686
3,Bandung Kulon,Cigondewah Rahayu,-6.94889,107.56314,683
4,Bandung Kulon,Caringin,-6.92727,107.57698,702


In [11]:
df_merged = pd.merge(new_df, coordinates, how='left', on='Kelurahan')
df_merged.dropna(inplace=True)
df_merged.drop("Kecamatan_y", axis=1, inplace=True)
df_merged.columns = ['Kecamatan','Kelurahan','Latitude','Longitude','Height']
df_merged.drop("Height", axis=1, inplace=True)
df_merged.head()

,Kecamatan,Kelurahan,Latitude,Longitude
0,Andir,Campaka,-6.89787,107.56314
1,Andir,Ciroyom,-6.91295,107.58617
2,Andir,Garuda,-6.91596,107.57656
3,Andir,Kebonjeruk,-6.91908,107.60107
4,Andir,Maleber,-6.90734,107.57344


In [14]:
address = 'Bandung, Indonesia'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bandung are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bandung are -6.9344694, 107.6049539.


In [18]:
# create map of Toronto using latitude and longitude values
map_bandung = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, kelurahan, kecamatan in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Kelurahan'], df_merged['Kecamatan']):
    label = '{}, {}'.format(kelurahan, kecamatan)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bandung)  
    
map_bandung

In [19]:
CLIENT_ID = 'WL3I3GC2FHSNC5UGPMYC3C5L1KMAFZPXAZCTMBRPZBBWGADA'
CLIENT_SECRET = 'J1VJFCE4UWUFRP550352VT1CWWANOV0Q21VJOJY0WLTJNAQU'
VERSION = '20200530'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WL3I3GC2FHSNC5UGPMYC3C5L1KMAFZPXAZCTMBRPZBBWGADA
CLIENT_SECRET:J1VJFCE4UWUFRP550352VT1CWWANOV0Q21VJOJY0WLTJNAQU


In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000, LIMIT=100):
    search_query = 'coffee'
    venues_list=[]
    for names, lat, lng in zip(names, latitudes, longitudes): 
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, search_query, radius, LIMIT)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except:
            print("group error", requests.get(url).json())
            print("-------------------------------------------")
        
        # return only relevant information for each nearby venue
        try:
            venues_list.append([(names, lat, lng, 
                    v['venue']['name'], 
                    v['venue']['location']['lat'], 
                    v['venue']['location']['lng'],  
                    v['venue']['categories'][0]['name']) for v in results])
        except:
            print("empty result")
            print("-------------------------------------------")
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Kelurahan', 
                  'Latitude Kelurahan', 
                  'Longitude Kelurahan', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
bandung_venues = getNearbyVenues(
    names=df_merged['Kelurahan'],
    latitudes=df_merged['Latitude'],
    longitudes=df_merged['Longitude']
)

In [38]:
bandung_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Campaka,-6.89787,107.56314,Coffee Toffee,-6.895865,107.558712,Coffee Shop
1,Campaka,-6.89787,107.56314,Warunk Upnormal,-6.892564,107.558444,Café
2,Campaka,-6.89787,107.56314,Dunkin',-6.890836,107.564225,Donut Shop
3,Campaka,-6.89787,107.56314,Sudi Mampir Cimindi,-6.894701,107.559498,Coffee Shop
4,Campaka,-6.89787,107.56314,Gurilaps Garut,-6.904773,107.564387,Café


In [47]:
# create map of Toronto using latitude and longitude values
map_cafe = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, venue, kategori, kelurahan in zip(bandung_venues['Venue Latitude'], bandung_venues['Venue Longitude'], bandung_venues['Venue'], bandung_venues['Venue Category'], bandung_venues['Neighborhood']):
    label = '{}, {}, {}'.format(venue, kategori, kelurahan)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cafe)  

hm_data = bandung_venues[["Venue Latitude", "Venue Longitude"]]
map_cafe.add_child(plugins.HeatMap(hm_data))
map_cafe

In [32]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude, 500, 100)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5edf63c0edbcad002348c8c3'},
 'response': {'headerLocation': 'Bandung',
  'headerFullLocation': 'Bandung',
  'headerLocationGranularity': 'city',
  'totalResults': 7,
  'suggestedBounds': {'ne': {'lat': -6.929969395499996,
    'lng': 107.6094786011752},
   'sw': {'lat': -6.938969404500004, 'lng': 107.6004291988248}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c5e067d85a1e21ebd4a5b11',
       'name': 'Monumen Bandung Lautan Api',
       'location': {'address': 'Lapangan Tegalega',
        'crossStreet': 'Jl. Otto Iskandardinata',
        'lat': -6.934177020485812,
        'lng': 107.60492313073063,
        'labeledLatLngs': [{'label': 'display',
          'lat': -6.934177020485812,
          'lng': 107.60492313073063}],
        'd